In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import optuna
from optuna.integration import TFKerasPruningCallback

In [2]:
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 40
N_TRAIN_EXAMPLES = 3000
STEPS_PER_EPOCH = int(N_TRAIN_EXAMPLES / BATCHSIZE / 10)
VALIDATION_STEPS = 30

In [3]:
def train_dataset():

    ds = tfds.load("mnist", split=tfds.Split.TRAIN, shuffle_files=True)
    ds = ds.map(lambda x: (tf.cast(x["image"], tf.float32) / 255.0, x["label"]))
    ds = ds.repeat().shuffle(1024).batch(BATCHSIZE)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)

    return ds

def eval_dataset():

    ds = tfds.load("mnist", split=tfds.Split.TEST, shuffle_files=False)
    ds = ds.map(lambda x: (tf.cast(x["image"], tf.float32) / 255.0, x["label"]))
    ds = ds.repeat().batch(BATCHSIZE)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)

    return ds

In [4]:
def create_model(trial):

    # Hyperparameters to be tuned by Optuna.
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    momentum = trial.suggest_float("momentum", 0.0, 1.0)
    units = trial.suggest_categorical("units", [32, 64, 128, 256, 512])

    # Compose neural network with one hidden layer.
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=units, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(CLASSES, activation=tf.nn.softmax))

    # Compile model.
    model.compile(
        optimizer=tf.keras.optimizers.SGD(lr=lr, momentum=momentum, nesterov=True),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

In [5]:
def objective(trial):
    # Clear clutter from previous TensorFlow graphs.
    tf.keras.backend.clear_session()

    # Metrics to be monitored by Optuna.
    if tf.__version__ >= "2":
        monitor = "val_accuracy"
    else:
        monitor = "val_acc"

    # Create tf.keras model instance.
    model = create_model(trial)

    # Create dataset instance.
    ds_train = train_dataset()
    ds_eval = eval_dataset()

    # Create callbacks for early stopping and pruning.
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=3),
        TFKerasPruningCallback(trial, monitor),
    ]

    # Train model.
    history = model.fit(
        ds_train,
        epochs=EPOCHS,
        steps_per_epoch=STEPS_PER_EPOCH,
        validation_data=ds_eval,
        validation_steps=VALIDATION_STEPS,
        callbacks=callbacks,
    )

    return history.history[monitor][-1]

In [6]:
def show_result(study):

    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [7]:
 study = optuna.create_study(
        direction="maximize", pruner=optuna.pruners.MedianPruner(n_startup_trials=2), study_name = "optuna_demo_study_01"
    )

[I 2021-01-25 11:37:14,176] A new study created in memory with name: optuna_demo_study_01


In [8]:
study.optimize(objective, n_trials=20, timeout=600)

Epoch 1/40
2/2 [==============================] - 0s 166ms/step - loss: 2.3678 - accuracy: 0.1016 - val_loss: 2.3388 - val_accuracy: 0.1365
Epoch 2/40
2/2 [==============================] - 0s 116ms/step - loss: 2.3491 - accuracy: 0.1445 - val_loss: 2.3329 - val_accuracy: 0.1393
Epoch 3/40
2/2 [==============================] - 0s 116ms/step - loss: 2.2807 - accuracy: 0.1719 - val_loss: 2.3280 - val_accuracy: 0.1440
Epoch 4/40
2/2 [==============================] - 0s 110ms/step - loss: 2.3507 - accuracy: 0.1055 - val_loss: 2.3221 - val_accuracy: 0.1448
Epoch 5/40
2/2 [==============================] - 0s 110ms/step - loss: 2.3275 - accuracy: 0.1484 - val_loss: 2.3165 - val_accuracy: 0.1466
Epoch 6/40
2/2 [==============================] - 0s 109ms/step - loss: 2.3055 - accuracy: 0.1484 - val_loss: 2.3119 - val_accuracy: 0.1508
Epoch 7/40
2/2 [==============================] - 0s 114ms/step - loss: 2.3268 - accuracy: 0.1445 - val_loss: 2.3070 - val_accuracy: 0.1534
Epoch 8/40
2/2 [====

[I 2021-01-25 11:37:25,448] Trial 0 finished with value: 0.22552083432674408 and parameters: {'lr': 0.002741400457556893, 'momentum': 0.15695769147814376, 'units': 32}. Best is trial 0 with value: 0.22552083432674408.


Epoch 1/40
2/2 [==============================] - 0s 146ms/step - loss: 2.3783 - accuracy: 0.1016 - val_loss: 2.3632 - val_accuracy: 0.0969
Epoch 2/40
2/2 [==============================] - 0s 111ms/step - loss: 2.4198 - accuracy: 0.0898 - val_loss: 2.3473 - val_accuracy: 0.0995
Epoch 3/40
2/2 [==============================] - 0s 113ms/step - loss: 2.3453 - accuracy: 0.0977 - val_loss: 2.3331 - val_accuracy: 0.1029
Epoch 4/40
2/2 [==============================] - 0s 112ms/step - loss: 2.3054 - accuracy: 0.1133 - val_loss: 2.3200 - val_accuracy: 0.1115
Epoch 5/40
2/2 [==============================] - 0s 108ms/step - loss: 2.3099 - accuracy: 0.0938 - val_loss: 2.3062 - val_accuracy: 0.1167
Epoch 6/40
2/2 [==============================] - 0s 107ms/step - loss: 2.3013 - accuracy: 0.1016 - val_loss: 2.2929 - val_accuracy: 0.1227
Epoch 7/40
2/2 [==============================] - 0s 107ms/step - loss: 2.3053 - accuracy: 0.1016 - val_loss: 2.2798 - val_accuracy: 0.1281
Epoch 8/40
2/2 [====

[I 2021-01-25 11:37:35,225] Trial 1 finished with value: 0.4559895694255829 and parameters: {'lr': 0.002416511777162246, 'momentum': 0.4067140265100664, 'units': 256}. Best is trial 1 with value: 0.4559895694255829.


Epoch 1/40
2/2 [==============================] - 0s 153ms/step - loss: 2.4013 - accuracy: 0.1250 - val_loss: 2.3752 - val_accuracy: 0.1516
Epoch 2/40
2/2 [==============================] - 0s 107ms/step - loss: 2.3781 - accuracy: 0.1445 - val_loss: 2.3713 - val_accuracy: 0.1521
Epoch 3/40
2/2 [==============================] - 0s 107ms/step - loss: 2.3540 - accuracy: 0.1445 - val_loss: 2.3674 - val_accuracy: 0.1534
Epoch 4/40
2/2 [==============================] - 0s 107ms/step - loss: 2.3697 - accuracy: 0.1211 - val_loss: 2.3635 - val_accuracy: 0.1555
Epoch 5/40
2/2 [==============================] - 0s 116ms/step - loss: 2.3803 - accuracy: 0.1094 - val_loss: 2.3593 - val_accuracy: 0.1570
Epoch 6/40
2/2 [==============================] - 0s 108ms/step - loss: 2.3488 - accuracy: 0.1680 - val_loss: 2.3554 - val_accuracy: 0.1586
Epoch 7/40
2/2 [==============================] - 0s 105ms/step - loss: 2.4002 - accuracy: 0.1445 - val_loss: 2.3512 - val_accuracy: 0.1599
Epoch 8/40
2/2 [====

...

Epoch 1/40


2/2 [==============================] - 0s 149ms/step - loss: 2.2873 - accuracy: 0.1211 - val_loss: 2.1795 - val_accuracy: 0.2120
Epoch 2/40
2/2 [==============================] - 0s 110ms/step - loss: 2.1570 - accuracy: 0.2344 - val_loss: 2.0299 - val_accuracy: 0.3581
Epoch 3/40
2/2 [==============================] - 0s 107ms/step - loss: 2.0096 - accuracy: 0.4023 - val_loss: 1.8982 - val_accuracy: 0.5016
Epoch 4/40
2/2 [==============================] - 0s 107ms/step - loss: 1.8594 - accuracy: 0.5586 - val_loss: 1.7724 - val_accuracy: 0.5802
Epoch 5/40
2/2 [==============================] - 0s 121ms/step - loss: 1.8006 - accuracy: 0.5625 - val_loss: 1.6591 - val_accuracy: 0.6445
Epoch 6/40
2/2 [==============================] - 0s 111ms/step - loss: 1.5688 - accuracy: 0.7227 - val_loss: 1.5521 - val_accuracy: 0.6456
Epoch 7/40
2/2 [==============================] - 0s 107ms/step - loss: 1.5771 - accuracy: 0.5938 - val_loss: 1.4517 - val_accuracy: 0.6823
Epoch 8/40
2/2 [===============

[I 2021-01-25 11:38:23,547] Trial 18 pruned. Trial was pruned at epoch 10.


Epoch 1/40
1/2 [==============>...............] - ETA: 0s - loss: 2.4565 - accuracy: 0.0547

[I 2021-01-25 11:38:24,387] Trial 19 pruned. Trial was pruned at epoch 0.


In [9]:
show_result(study)

Study statistics: 
  Number of finished trials:  20
  Number of pruned trials:  15
  Number of complete trials:  5
Best trial:
  Value:  0.8791666626930237
  Params: 
    lr: 0.0901486222345401
    momentum: 0.26264387865128735
    units: 512


In [11]:
optuna.visualization.plot_optimization_history(study)